# Vizualizace
Pojďme si načíst výsledky z webu sreality, a pokusit se najít vztahy mezi bydlením

In [50]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [51]:
file = "flats.csv"
df = pd.read_csv("../data/flats.csv", index_col=0)
df.shape

(11469, 17)

Pojďme se kouknout na základní statistiky našeho datového souboru

In [52]:
df.head(1)

,region,Cena,Typ_ceny,api_detail,Dispozice,typ,Stavba,Stav objektu,Vlastnictví,Podlaží,Užitná plocha,Plocha podlahová,Balkón,Energetická náročnost budovy,Výtah,Lodžie,Terasa
0,Jihočeský kraj,5179000.0,total,https://www.sreality.cz/api/cs/v2/estates/2419...,3+1,Byt,Cihlová,Velmi dobrý,Osobní,3.0,118,108.0,9.0,G,NaN,NaN,NaN


In [53]:
df.describe()

,Cena,Podlaží,Užitná plocha,Plocha podlahová,Balkón,Výtah,Lodžie,Terasa
count,1.070200e+04,11454.000000,11469.000000,6211.000000,2430.000000,7948.000000,2579.000000,742.000000
mean,6.218969e+06,3.457482,59.250763,59.788440,2.382305,0.658782,2.482358,16.696765
std,5.061600e+06,2.834483,56.847670,41.927992,3.329190,0.474148,2.667939,23.979545
min,2.530000e+05,1.000000,8.000000,4.000000,1.000000,0.000000,1.000000,1.000000
25%,3.199000e+06,2.000000,34.000000,35.000000,1.000000,0.000000,1.000000,1.000000
50%,4.823000e+06,3.000000,46.000000,47.000000,1.000000,1.000000,1.000000,8.000000
75%,7.767500e+06,4.000000,74.000000,77.000000,2.000000,1.000000,3.000000,22.750000
max,9.538400e+07,170.000000,4084.000000,1255.000000,46.000000,1.000000,21.000000,193.000000


In [54]:
df["Vlastnictví"].unique()

array(['Osobní', 'Družstevní', 'Státní/obecní'], dtype=object)

Zaměříme se pouze na byty v osobním vlastnictví

In [55]:
df = df[df["Vlastnictví"] == "Osobní"]
df.shape

(9574, 17)

Přišli jsme zhruba o dva tisíce řádků

# Složení datového souboru
Pojďme se kouknout, jaké byty tvoří náš datový soubor

In [59]:
structure = df.copy()
structure["Dispozice"] = structure["Dispozice"].astype(str)
structure = structure[structure["Dispozice"] != "nan"]
structure["Počet obytných místností"] = [value[0] for value in structure["Dispozice"]]

grouped = structure.groupby("region").count()["api_detail"].sort_values(ascending=False)

meta = structure.pivot_table(
    values=["api_detail"],
    index=["region"], 
    columns=["Počet obytných místností"],
    aggfunc="count"
).reset_index()
meta.columns = ["region", "1", "2", "3", "4", "5"]

fig = px.bar(
    meta,
    x="region",
    y=["1", "2", "3", "4", "5"],
    labels={"value":"Počet jednotek", "region":"Region"},
    title="Dostupné jednotky dle krajů",
    category_orders = {
        "region": grouped.index
    },
    barmode="stack",
)
fig.update_layout(
    legend_title="Počet místností",
)
fig.show()
fig.write_image("./images/dostupne_jednotky_dle_kraju.png")

Nejvíce nabízených jednotek má dvě nebo tři obytné místnosti

# Boxplot
Pojďme se kouknout na rozpětí cen bytů pro různé dispozice
Uvažují se byty v soukromém vlastnictví

In [61]:
boxdata = df.copy()
boxdata["Dispozice"] = boxdata["Dispozice"].astype(str)
boxdata = boxdata[boxdata["Dispozice"] != "nan"]

fig = px.box(
    boxdata,
    x="Dispozice",
    y="Cena",
    color="Dispozice",
    title="Rozpětí cen bytů pro různé dispozice",
    range_y = [0, 40000000],
    category_orders = {"Dispozice": ["1+kk", "1+1", "2+kk", "2+1", "3+kk", "3+1", "4+kk", "4+1", "5+kk", "5+1"]}
)
fig.show()
fig.write_image("./images/cenove_rozpeti_dle_dispozice.png")

Překvapivě jsou byty typu +1 (1+1, 2+1, etc.) průměrně levnější, než teoreticky menší byty +kk (1+kk, 2+kk, etc.)<br>
Je možné, že se tyto byty nachází ve starších budovách, a proto jsou méně žádané (tím pádem i levnější)

Pojďme si přidat sloupec počet místností a kouknout se, jak roste cena v závislosti na metráži

In [62]:
scatterdata = df.copy()
scatterdata["Dispozice"] = scatterdata["Dispozice"].astype(str)
scatterdata = scatterdata[scatterdata["Dispozice"] != "nan"]
scatterdata["Počet obytných místností"] = [value[0] for value in scatterdata["Dispozice"]]

fig = px.scatter(
    scatterdata,
    x="Užitná plocha",
    y="Cena",
    color="Počet obytných místností",
    # facet_row="Počet místností",
    title="Cena vs užitná plocha",
    range_x = [0, 200],
    range_y = [0, 20000000],
    # trendline="ols",
    category_orders = {"Počet obytných místností": ["1", "2", "3", "4", "5"]}
)
fig.show()
fig.write_image("./images/cena_vs_uzitna_plocha_scatter.png")

V tomto grafu vidíme, že je existuje slabý vztah mezi užitnou plochou bytu a jeho cenou, avšak se to nedá potvrdit jednoznačně.

# Dostupnost
Pojďme si zjistit průměrnou cenu bytu 2+kk v různých krajích

In [63]:
df.columns

Index(['region', 'Cena', 'Typ_ceny', 'api_detail', 'Dispozice', 'typ',
       'Stavba', 'Stav objektu', 'Vlastnictví', 'Podlaží', 'Užitná plocha',
       'Plocha podlahová', 'Balkón', 'Energetická náročnost budovy', 'Výtah',
       'Lodžie', 'Terasa'],
      dtype='object')

In [64]:
df["Stav objektu"].unique()

array(['Velmi dobrý', 'Dobrý', 'Před rekonstrukcí', 'Po rekonstrukci',
       'Špatný', 'Ve výstavbě', 'Projekt'], dtype=object)

In [66]:
boxdata2 = df.copy()
scatterdata["Dispozice"] = scatterdata["Dispozice"].astype(str)
boxdata2 = boxdata2[boxdata2["Dispozice"] == "2+kk"]
boxdata2 = boxdata2.dropna(axis=0, subset=["Cena", "Užitná plocha"])
boxdata2["Cena za m2"] = boxdata2["Cena"] / boxdata2["Užitná plocha"]

grouped = boxdata2.groupby("region").median()["Cena"].sort_values(ascending=False)

fig = px.box(
    boxdata2,
    x="region",
    y="Cena",
    color="region",
    title="Cena za byt 2+kk v různých krajích",
    range_y = [0, 15000000],
    category_orders = {"region": grouped.index}
)
fig.show()
fig.write_image("./images/cena_2kk_dle_kraju.png")

Zde vidíme, že nejdražší byty se nachází v Praze, zatímco nejlevnější jsou v Ústeckém kraji.

# Platy
Nyní pojďme přidat data o průměrných výdělcích z ISPV. Data jsem musel předělat do formátu csv, aby se s ním dalo pracovat v pythonu

In [67]:
by_region = pd.read_excel("../data/ISPV.xlsx", sheet_name="regiony")

In [68]:
print(by_region)

                Region  Medián  Průměr    1.D    1.Q    3.Q    9.D
0   Hlavní město Praha   39446   50363  19788  28270  56859  89146
1          Středočeský   35054   40328  19273  25592  47306  64435
2            Jihočeský   31731   36818  19618  24766  41640  57179
3             Plzeňský   34257   37976  20248  26454  43339  57096
4          Karlovarský   29752   33957  19200  23867  38992  50645
5              Ústecký   32639   37220  18822  25197  42634  57142
6            Liberecký   32566   36964  20275  25441  42077  55911
7      Královéhradecký   33396   37480  20147  26255  43192  58132
8           Pardubický   32028   35752  19922  24976  41295  53149
9             Vysočina   32427   36131  19499  25376  41481  52874
10        Jihomoravský   33547   39191  19725  25768  43949  62308
11           Olomoucký   31189   35178  19242  24286  40326  53245
12             Zlínský   31750   35832  19432  24543  41696  54053
13     Moravskoslezský   31744   35655  19198  24320  40935  5

# Kraje
* Pojďme zjistit délku splácení hypotéky při mediánovém platu v daném kraji<br>
    * Budeme počítat, že osoba může alokovat maximálně 40% hrubého platu na splácení hypotéky<br>
        * (Zhruba 80% hrubé mzdy zbyde jako čistá mzda dle této [kalkulačky](https://www.penize.cz/kalkulacky/vypocet-ciste-mzdy#mzda))<br>
* Zároveň si lze pořídit hypotéku pouze na 80% hodnoty nemovitosti<br>
    * (ve ojedinělých případech i 90%, ale v této situaci se vyplatí být konzervativní)

In [69]:
hypoteka = df.copy()
hypoteka = hypoteka[hypoteka["Cena"] > 100000]
hypoteka["Dispozice"] = hypoteka["Dispozice"].astype(str)
hypoteka["Počet obytných místností"] = [value[0] for value in hypoteka["Dispozice"]]
hypoteka = hypoteka[hypoteka["Počet obytných místností"] == "2"]
hypoteka = hypoteka.groupby("region").median()["Cena"].reset_index()
hypoteka["Výše poskytnuté hypotéky"] = hypoteka["Cena"]*0.8

In [70]:
def monthly_payment(outstanding_value:int, interest_rate:float, payments_per_year:int, number_of_years:int):
    monthly_payment = outstanding_value * ((interest_rate/payments_per_year)*(1+(interest_rate/payments_per_year))**(payments_per_year*number_of_years)) / ((1+(interest_rate/payments_per_year))**(payments_per_year*number_of_years)-1)
    return round(monthly_payment, 0)

In [71]:
hypoteka["Měsíční splátka"] = [monthly_payment(value, 0.053, 12, 30) for value in hypoteka["Výše poskytnuté hypotéky"]]

In [72]:
by_region["Medián čisté mzdy"] = by_region["Medián"] * 0.8
by_region["Region"] = ["Praha"] + [f"{region} kraj" if region not in ["Vysočina"] else region for region in by_region["Region"][1:]]
by_region = by_region.rename(columns= {"Region":"region"}, inplace=False)

In [73]:
waterfalldata = by_region.copy()
waterfalldata = waterfalldata.merge(hypoteka, how="outer", on="region")
waterfalldata["50% čisté mzdy"] = waterfalldata["Medián čisté mzdy"] * 0.5
waterfalldata["saldo"] = round(waterfalldata["Medián čisté mzdy"] - waterfalldata["Měsíční splátka"], 0)
waterfalldata["Barva"] = np.where(waterfalldata["saldo"]<0, 'red', 'green')

grouped = waterfalldata.sort_values(by="saldo", ascending=False)

fig = px.bar(
    waterfalldata,
    x="region",
    y="saldo",
    color="saldo",
    title="Měsíční saldo čisté mzdy a splátky hypotéky pro byt se dvěmi obytnými místnostmi",
    labels={"saldo": "Saldo rozpočtu (v kč)", "region": "Region"},
    category_orders = {"region": grouped["region"]}
)
fig.show()
fig.write_image("./images/saldo_ciste_mzdy_a_hypoteky.png")

In [74]:
waterfalldata

,region,Medián,Průměr,1.D,1.Q,3.Q,9.D,Medián čisté mzdy,Cena,Výše poskytnuté hypotéky,Měsíční splátka,50% čisté mzdy,saldo,Barva
0,Praha,39446,50363,19788,28270,56859,89146,31556.8,8210000.0,6568000.0,36472.0,15778.4,-4915.0,red
1,Středočeský kraj,35054,40328,19273,25592,47306,64435,28043.2,4817000.0,3853600.0,21399.0,14021.6,6644.0,green
2,Jihočeský kraj,31731,36818,19618,24766,41640,57179,25384.8,4105000.0,3284000.0,18236.0,12692.4,7149.0,green
3,Plzeňský kraj,34257,37976,20248,26454,43339,57096,27405.6,4225000.0,3380000.0,18769.0,13702.8,8637.0,green
4,Karlovarský kraj,29752,33957,19200,23867,38992,50645,23801.6,3018000.0,2414400.0,13407.0,11900.8,10395.0,green
5,Ústecký kraj,32639,37220,18822,25197,42634,57142,26111.2,2294000.0,1835200.0,10191.0,13055.6,15920.0,green
6,Liberecký kraj,32566,36964,20275,25441,42077,55911,26052.8,3893500.0,3114800.0,17297.0,13026.4,8756.0,green
7,Královéhradecký kraj,33396,37480,20147,26255,43192,58132,26716.8,4213000.0,3370400.0,18716.0,13358.4,8001.0,green
8,Pardubický kraj,32028,35752,19922,24976,41295,53149,25622.4,4793000.0,3834400.0,21293.0,12811.2,4329.0,green
9,Vysočina,32427,36131,19499,25376,41481,52874,25941.6,3975000.0,3180000.0,17659.0,12970.8,8283.0,green


Zde vidíme, že ve většině krajů není mediánový byt 2+kk dostupný pro osobu s mediánovým platem<br>
Nejhůře je na tom Praha, kde by mediánový zaměstnanec potřeboval zhruba 4,9 tisíce, aby dosáhl na splátku<br>
Nejlépe je na tom Ústecký kraj, kde zbyde zaměstnanci více než 15,9 tisíce po zaplacení měsíční splátky hypotéky<br>
Toto číslo ovšem může být zavádějící, protože je v Ústeckém kraji málo dostupných bytů

# Dostupnost bydlení dle profese
Pojďme se kouknout na schopnost pořídit si vlastní bydlení dle zaměstnání

In [75]:
by_job_type = pd.read_excel("../data/ISPV.xlsx", sheet_name="odvetvi")

In [76]:
by_job_type.head(5)

,Odvětví,Medián,Průměr,1.D,1.Q,3.Q,9.D
0,Řídící pracovníci,70660,93030,33174,46315,110499,172238
1,Nejvyšší představitelé společností,92426,146790,33855,50097,182075,330418
2,"Řídící pracovníci správy podniku, obchod., adm...",83295,104733,37937,53516,126569,187578
3,"Řídící pracovníci výroby, IT, vzdělávání a v p...",70902,89365,35579,48442,106349,162920
4,"Řídící pracovníci ubyt., strav. služeb, obchod...",45787,61580,25065,34035,69983,111934


In [77]:
def mortgage_length(outstanding_value:int, interest_rate:float, payments_per_year:int, available_funds:int):
    num_years = 1
    while True:
        if num_years > 50:
            return 50
        else:
            mon_pay = monthly_payment(outstanding_value, interest_rate, payments_per_year, num_years)
            if mon_pay < available_funds:
                break
            else:
                num_years += 1
    return num_years

In [78]:
selected_flat_sizes = ["1+kk", "2+kk", "3+kk"]
flatdata = df.copy()
flatdata = flatdata[flatdata["Dispozice"].isin(selected_flat_sizes)]
flatdata = flatdata.groupby("Dispozice").median()["Cena"].reset_index()
flatdata["Výše hypotéky"] = flatdata["Cena"] * 0.8

pointdata = by_job_type.copy()
pointdata["Medián čisté mzdy"] = pointdata["Medián"]*0.8
pointdata["50% čisté mzdy"] = pointdata["Medián čisté mzdy"]*0.5
for disposition in selected_flat_sizes:
    pointdata[f"Doba splácení {disposition}"] = [
        mortgage_length(
            int(flatdata["Výše hypotéky"].loc[flatdata["Dispozice"] == disposition]),
            0.053,
            12,
            pay
        )
        for pay in pointdata["Medián čisté mzdy"]
]
pointdata = pointdata.sort_values(by="Medián", ascending=False)

In [79]:
fig = go.Figure(
    go.Scatter(name="1+kk", x=pointdata["Odvětví"], y=pointdata["Doba splácení 1+kk"], line_shape="hvh")
)
fig.add_trace(
    go.Scatter(name="2+kk", x=pointdata["Odvětví"], y=pointdata["Doba splácení 2+kk"], line_shape="hvh")
)
fig.add_trace(
    go.Scatter(name="3+kk", x=pointdata["Odvětví"], y=pointdata["Doba splácení 3+kk"], line_shape="hvh")
)
fig.update_layout(
    title_text="Hypotetická doba splácení hypotéky pro různé dispozice a profese<br>"
    "<sup>Za situace, kdy zaměstnanec dané skupiny s mediánovým příjmem dá celý čistý plat na splácení hypotéky</sup>"
)
fig.update_xaxes(visible=False)
fig.update_yaxes(title="Doba splácení v letech", range=[5, 40])
fig.add_hline(
    y=30,
    line_dash="dot",
    annotation_text="30 let",
    annotation_position="bottom right"
)
fig.add_hrect(
    y0=30,
    y1=50,
    line_width=0,
    fillcolor="red",
    opacity=0.1
)
fig.add_annotation(
    x="Specialisté v oblasti vědy a techniky",
    y=31,
    xref="x",
    yref="y",
    text="Zaměstnanec si není schopný vzít hypotéku",
    bgcolor="white",
    showarrow=False,
    opacity=0.8
)
fig.add_annotation(
    x="Řídící pracovníci",
    y=15,
    xref="x",
    yref="y",
    text="Vedoucí",
    showarrow=True,
    xshift=-9,
    ax=-15,
    ay=-34
)
fig.add_annotation(
    x="Specialisté",
    y=23,
    xref="x",
    yref="y",
    text="Specialisté",
    showarrow=True,
    xshift=-9,
    ax=-30,
    ay=-30
)
fig.add_annotation(
    x="Kovodělníci, strojírenští dělníci a pracovníci v příbuzných oborech",
    y=15,
    xref="x",
    yref="y",
    text="Odborní dělníci",
    showarrow=True,
    xshift=-9,
    ax=-40,
    ay=-20
)
fig.add_annotation(
    x="Odborní pracovníci v oblasti zdravotnictví",
    y=24,
    xref="x",
    yref="y",
    text="Odborní zdravotníci",
    showarrow=True,
    xshift=-9,
    ax=-19,
    ay=-30
)
fig.add_annotation(
    x="Řidiči a obsluha pojízdných zařízení",
    y=17,
    xref="x",
    yref="y",
    text="Řidiči",
    showarrow=True,
    xshift=-9,
    ax=-20,
    ay=-25
)
fig.add_annotation(
    x="Úředníci",
    y=19,
    xref="x",
    yref="y",
    text="Úředníci",
    showarrow=True,
    xshift=-9,
    ax=-20,
    ay=-20
)
fig.add_annotation(
    x="Pracovníci v oblasti uměleckých a tradičních řemesel a polygrafie",
    y=21,
    xref="x",
    yref="y",
    text="Řemeslníci",
    showarrow=True,
    xshift=-9,
    ax=-30,
    ay=-20
)
fig.add_annotation(
    x="Pracovníci ve službách a prodeji",
    y=27,
    xref="x",
    yref="y",
    text="Služby a prodeje",
    showarrow=True,
    xshift=-9,
    ax=-40,
    ay=-15
)
fig.add_annotation(
    x="Pomocní a nekvalifikovaní pracovníci",
    y=35,
    xref="x",
    yref="y",
    text="Pomocné profese",
    showarrow=True,
    xshift=-9,
    ax=-50,
    ay=-30
)
fig.show()
fig.write_image("./images/hypoteticka_doba_splaceni.png")